In [ ]:
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
# gpt_output = pd.read_csv('train_drug_label_gpt35_output_uog.csv')
folder = os.getcwd() + '/'
gpt_output = pd.read_csv(folder + 'train_drug_label_gpt35_finetuned_output_3.csv')
manual_ades = pd.read_csv(folder + 'train_drug_label_text_manual_ades.csv')
# manual_ades = pd.read_csv('train_manual_ades.csv')
# test_df = pd.read_csv(folder + 'train_drug_label_text_subsections.csv')
# manual_ades = test_df[test_df['section_name'] == 'adverse reactions']

In [ ]:
gpt_output.head()

In [ ]:
# Evaluation 

count = 0
results = list()
gpt_model = "gpt4"
for drug in tqdm(drugs):
    drug_df = manual_ades[manual_ades['drug_name'] == drug]
    manual = set(drug_df['reaction_string'].str.lower())
    gpt_drug = (gpt_output[
        (gpt_output['drug_name'] == drug)
        &
        ((gpt_output['section_name'] == "adverse reactions"))
        ][gpt_model + "_output"].astype(str)
        .str.lower()
        .str.replace('\n-', ', ')
        .str.split(",").tolist())
    try:
        gpt_drug = [item.strip() for row in gpt_drug for item in row]
        gpt_drug = [item for item in gpt_drug if len(item) <= 71]
        # gpt_drug = [x.strip() for x in gpt_drug[0]]
        gpt_drug = set(gpt_drug)
    except:
        continue

    TP = len(manual.intersection(gpt_drug))
    FP = len(gpt_drug.difference(manual))
    FN = len(manual.difference(gpt_drug))
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    if (precision + recall) != 0 :
        f1 = (2 * precision * recall)/(precision + recall)# 2*TP/(2*TP+FP+FN)
    else:
        f1 = 0

    results.append([drug, len(manual), len(gpt_drug), TP, FP, FN, precision, recall, f1])
results = pd.DataFrame(results, columns=['drug_name', 'n_manual', 'n_gpt', 'tp', 'fp', 'fn', 'precision', 'recall', 'f1'])
results.head()

In [ ]:
[tp_total, fp_total, fn_total] =  results[['tp', 'fp', 'fn']].sum()
precision = tp_total/(tp_total+fp_total)
recall = tp_total/(tp_total+fn_total)
f1 = 2*tp_total/(2*tp_total+fp_total+fn_total)
print("Results")
print("precision: {}\nrecall: {}\nf1: {}".format(precision, recall, f1))  

In [ ]:
results_filt = results[results['f1'] > 0]
results_filt.head()
print("Number of drugs with f1 > 0:", len(results_filt))
print("Number of drugs:", len(results))

In [ ]:
[tp_total, fp_total, fn_total] =  results_filt[['tp', 'fp', 'fn']].sum()
precision = tp_total/(tp_total+fp_total)
recall = tp_total/(tp_total+fn_total)
f1 = 2*tp_total/(2*tp_total+fp_total+fn_total)
print("FILTERED RESULTS")
print("precision: {}\nrecall: {}\nf1: {}".format(precision, recall, f1))  

In [ ]:
results.to_csv("finetuned_results.csv")

In [ ]:
results_top = results.sort_values('recall')
bottom = results_top[:10].drug_name.to_list()
bottom

In [ ]:
results = pd.read_csv('finetuned_results.csv')
results_2 = results[results['f1'] > 0]
results_2.sort_values('f1')

In [ ]:
### Test one drug 
# gpt_output = []
# drug_df = manual_ades[manual_ades['drug_name'] == "FARXIGA"]
# manual = set(drug_df['reaction_string'].str.lower())
# manual

In [ ]:
drug = 'FARXIGA'
TP = len(manual.intersection(gpt_output))
FP = len(gpt_output.difference(manual))
FN = len(manual.difference(gpt_output))
if TP + FP == 0:
        precision = 0
else:
    precision = TP/(TP+FP)
recall = TP/(TP+FN)
if (precision + recall) != 0 :
    f1 = (2 * precision * recall)/(precision + recall)# 2*TP/(2*TP+FP+FN)
else:
    f1 = 0

In [ ]:
results_ind = list()
results_ind.append([drug, len(manual), len(gpt_output), TP, FP, FN, precision, recall, f1])
results_ind = pd.DataFrame(results_3, columns=['drug_name', 'n_manual', 'n_gpt', 'tp', 'fp', 'fn', 'precision', 'recall', 'f1'])
results_ind.head()

In [ ]:
# ! pip install matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Sample data: Generating 1000 data points from a normal distribution

# Creating the histogram
plt.hist(results_top['f1'], bins=30, color='black', alpha = 0.5)
plt.hist(results_top['recall'], bins=30, color='red', alpha = 0.5)
plt.hist(results_top['precision'], bins=30, color='blue', alpha = 0.5)



# Adding title and labels
plt.title('Precision // Recall // F1')
# plt.xlabel('Value')
# plt.ylabel('Frequency')

# Showing the plot
plt.show()


In [ ]:
prompt = "run this script {}"
text = "a + b"
print(prompt.format(text))